In [1]:
import numpy as np
import pandas as pd

In [2]:
# !curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
# conda_installer.install()
conda_installer.install()
# !/root/miniconda/bin/conda info -e
import rdkit
import deepchem as dc

add /home/ali/miniconda/lib/python3.6/site-packages to PYTHONPATH
all packages are already installed


In [3]:
pdb = rdkit.Chem.MolFromPDBFile('../Datasets/PDBBindSmallSet/1a30-ligand.pdb')

In [4]:
ligand = pd.read_csv('../Datasets/PDBBindSmallSet/csv-files/1a30-info.csv')

In [14]:
ligand

,Unnamed: 0,x,y,z,atomic-charge,atomic-radius,effective-born-radii
0,0,28.714,40.271,5.327,-0.2020,1.55,0.390811
1,1,29.274,41.102,5.453,0.3120,1.30,0.607797
2,2,27.976,40.469,4.658,0.3120,1.30,0.431638
3,3,28.138,39.883,6.633,-0.0120,1.70,0.339602
4,4,27.051,39.953,6.613,0.1000,1.20,0.286345
...,...,...,...,...,...,...,...
3183,3183,9.154,25.597,1.832,0.0195,1.20,0.168857
3184,3184,9.573,24.390,0.628,0.0195,1.20,0.164129
3185,3185,8.311,21.948,2.155,0.0195,1.20,0.187242
3186,3186,9.906,21.862,1.417,0.0195,1.20,0.214430


In [8]:
rdkit_order = None
for c in pdb.GetConformers():
    rdkit_order = c.GetPositions()

In [10]:
f = None
df = pd.DataFrame(columns=ligand.columns)
for i, atom_pos in enumerate(rdkit_order):
#     print(i, " ", atom_pos)
#     print(ligand['x'].iloc[0])
#     print(ligand[(ligand['x'] == atom_pos[0]) & (ligand['y'] == atom_pos[1]) & (ligand['z'] == atom_pos[2])])
    df = df.append(ligand[(ligand['x'] == atom_pos[0]) & (ligand['y'] == atom_pos[1]) & (ligand['z'] == atom_pos[2])], ignore_index=True)
#     break

In [11]:
featurizer = dc.feat.ConvMolFeaturizer(per_atom_fragmentation=False)

In [12]:
c_f = featurizer.featurize(pdb)[0]

In [27]:
import os
pdb_files = ["../Datasets/PDBBindSmallSet/" + pdb for pdb in os.listdir("../Datasets/PDBBindSmallSet") if pdb.__contains__('.pdb')]
atoms_csvs = ["../Datasets/PDBBindSmallSet/csv-files/" + atom_csv for atom_csv in os.listdir("../Datasets/PDBBindSmallSet/csv-files/") if atom_csv.__contains__('.csv')]
physics_df = pd.read_csv('../Datasets/PDBBindSmallSet/polished.csv')

In [28]:
physics_df = physics_df[physics_df['dataset group name'] == 'pdbbind']

In [33]:
mols_dict = {}
for pdb in pdb_files:
    mols_dict.update({pdb.split('/')[-1].split('.')[0].split('-')[0]: rdkit.Chem.MolFromPDBFile(pdb)})

RDKit WARNING: [19:26:57] Cannot determine element for PDB atom #3177
RDKit WARNING: [19:26:57] Cannot determine element for PDB atom #6016
RDKit WARNING: [19:27:47] Cannot determine element for PDB atom #3862
RDKit WARNING: [19:27:48] Cannot determine element for PDB atom #21503
RDKit WARNING: [19:27:56] Cannot determine element for PDB atom #3395
RDKit WARNING: [19:27:56] Cannot determine element for PDB atom #3160
RDKit WARNING: [19:27:56] Cannot determine element for PDB atom #5566
RDKit WARNING: [19:27:56] Cannot determine element for PDB atom #3174
RDKit WARNING: [19:27:56] Cannot determine element for PDB atom #13170


In [36]:
rdkit.Chem.MolFromPDBFile(pdb_files[0])

RDKit WARNING: [19:29:49] Cannot determine element for PDB atom #3177


In [34]:
mols_dict

{'1a94': None,
 '1a4r': None,
 '1a8i': <rdkit.Chem.rdchem.Mol at 0x7efee1fd9120>,
 '1a4w': None,
 '1a30': <rdkit.Chem.rdchem.Mol at 0x7efee2a74850>,
 '1a69': None,
 '1a99': <rdkit.Chem.rdchem.Mol at 0x7efee2ccba30>,
 '1a1e': None,
 '1a9m': None,
 '1add': None,
 '1aaq': None,
 '1a4k': None}